# COVID-19 Analysis & Visualization using Plotly Express

In [ ]:
import pandas as pdimport numpy as npimport plotly.express as pximport plotly.graph_objects as gourl = "https://covid.ourworldindata.org/data/owid-covid-data.csv"df = pd.read_csv(url)df.head()df['date'] = pd.to_datetime(df['date'])df_clean = df[['iso_code','continent','location','date',               'total_cases','new_cases','total_deaths','new_deaths',               'total_vaccinations','people_vaccinated','people_fully_vaccinated',               'population']]df_clean = df_clean.dropna(subset=['location'])latest = df_clean[df_clean['date'] == df_clean['date'].max()]global_summary = latest[['total_cases','total_deaths','people_fully_vaccinated']].sum()print("🌍 Global COVID-19 Summary:")print(global_summary)world = df_clean.groupby('date')['total_cases'].sum().reset_index()fig = px.line(world, x='date', y='total_cases', title='🌍 Global Total COVID-19 Cases Over Time')fig.show()countries = ['India', 'United States', 'Brazil', 'Russia', 'United Kingdom']df_countries = df_clean[df_clean['location'].isin(countries)]fig = px.line(df_countries, x='date', y='total_cases', color='location',              title='📈 Total Cases Comparison by Country')fig.show()top10 = latest.nlargest(10, 'total_cases')fig = px.bar(top10, x='location', y='total_cases',             title='🏆 Top 10 Countries by Total Cases', text='total_cases')fig.show()fig = px.scatter(latest, x='total_cases', y='total_deaths',                 color='continent', size='population',                 hover_name='location',                 title='⚖️ Relationship Between Cases & Deaths')fig.show()latest['cases_per_million'] = latest['total_cases'] / (latest['population']/1e6)latest['deaths_per_million'] = latest['total_deaths'] / (latest['population']/1e6)fig = px.scatter(latest,                 x='cases_per_million', y='deaths_per_million',                 size='population', color='continent',                 hover_name='location',                 title='🟠 Cases per Million vs Deaths per Million (Bubble Chart)')fig.show()vaccination = df_clean.groupby('date')['people_fully_vaccinated'].sum().reset_index()fig = px.line(vaccination,              x='date', y='people_fully_vaccinated',              title='💉 Global Fully Vaccinated Population Over Time')fig.show()cont_df = df_clean.groupby(['continent','date'])['new_cases'].sum().reset_index()fig = px.density_heatmap(cont_df,                         x='date', y='continent', z='new_cases',                         title='🔥 New Cases Heatmap by Continent')fig.show()from statsmodels.tsa.arima.model import ARIMAts = world.set_index('date')['total_cases']model = ARIMA(ts, order=(5,1,0))result = model.fit()forecast = result.predict(start=len(ts), end=len(ts)+30)fig = go.Figure()fig.add_trace(go.Scatter(x=ts.index, y=ts, name='Actual Cases'))fig.add_trace(go.Scatter(x=pd.date_range(ts.index[-1], periods=31, freq='D'),                         y=forecast, name='Forecasted Cases'))fig.update_layout(title='🔮 30-Day COVID-19 Forecast')fig.show()